In [2]:
from nltk.corpus import stopwords 
from collections import Counter
import re
import numpy as np
from datasets import load_dataset

imdb = load_dataset("imdb")

train_dataset=imdb['train']
test_dataset=imdb['test']

train_X=train_dataset['text']
train_Y=train_dataset['label']
test_X=test_dataset['text']
test_Y=test_dataset['label']

import warnings
warnings.simplefilter("ignore")

In [4]:
import nltk
#nltk.download('stopwords')
def preprocess(s):
    s = re.sub(r"[^\w\s]", '', s)
    s = re.sub(r"\s+", '', s)
    s = re.sub(r"\d", '', s)
    return s

def tockenize(train_X,test_X, vocab_size):
    word_list = []

    stop_words = set(stopwords.words('english')) 
    for sent in train_X:
        for word in sent.lower().split():
            word = preprocess(word)
            if word not in stop_words and word != '':
                word_list.append(word)
  
    counted_word_list = Counter(word_list)

    sorted_word_list= sorted(counted_word_list,key=counted_word_list.get,reverse=True)[:vocab_size-1]

    vocab = {w:i+1 for i,w in enumerate(sorted_word_list)}
    
    # tockenize
    final_list_train,final_list_test = [],[]
    for sent in train_X:
        final_list_train.append([vocab[preprocess(word)] for word in sent.lower().split() if preprocess(word) in vocab.keys()])
    for sent in test_X:
        final_list_test.append([vocab[preprocess(word)] for word in sent.lower().split()  if preprocess(word) in vocab.keys()])
            

    return np.array(final_list_train, dtype=object), np.array(final_list_test, dtype=object), vocab


In [6]:
vocab_size=1000
train_X, test_X, vocab = tockenize(train_X, test_X, vocab_size)

In [7]:
print(f'Length of vocabulary is {len(vocab)+1}')

Length of vocabulary is 1000


In [10]:
def padding_(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

In [11]:
seq_len=500
train_X_pad = padding_(train_X,seq_len)
test_X_pad = padding_(test_X,seq_len)

In [12]:
import json

with open(f'vocab_{vocab_size}.json', 'w') as file:
    json.dump(vocab, file)

In [13]:
np.savetxt(f'train_X_pad.txt', train_X_pad, fmt='%d')
np.savetxt(f'test_X_pad.txt', test_X_pad, fmt='%d')

In [14]:
np.savetxt(f'train_Y.txt', train_Y, fmt='%d')
np.savetxt(f'test_Y.txt', test_Y, fmt='%d')